In [1]:
import numpy as np

from src.vrp_study.data_loader import parse_data
from vrp_study.initial_solution.pdptw_solution_builder import PDPTWSolutionBuilder
from vrp_study.managers.pdptw_routing_manager_builder import PDRoutingManagerBuilder
from vrp_study.or_tools.model import *

In [2]:
benchmark_type = 'pdp_400'
name = 'LC1_4_9.txt'

In [3]:
depo, cargos, tariff = parse_data(f'../data/Li & Lim benchmark/benchmarks/{benchmark_type}/{name}')

In [4]:
p2coordinates = {}
p2coordinates.update({
    crg.nodes[i].id: crg.nodes[i].coordinates for crg in cargos for i in range(2)
})
p2coordinates[depo.id] = depo.coordinates
distance_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in
                   p2coordinates.items() for
                   v, dv in p2coordinates.items()}
time_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in p2coordinates.items() for
               v, dv in p2coordinates.items()}

In [5]:
len(cargos)

209

In [6]:
routing_manager = PDRoutingManagerBuilder(
    distance_matrix=distance_matrix,
    time_matrix=time_matrix,
).with_tariffs(tariff).with_cargos(cargos).with_depo(depo).build()

In [7]:
model_factory = VRPModelFactory.get_pdptw_model_factory(routing_manager)

In [8]:
callback = SolutionCallback()

In [9]:

initial_solution_builder = PDPTWSolutionBuilder(
    solver=VRPSolver(
        model_config=ModelConfig(max_execution_time_minutes=1, max_solution_number=50)
    ),
    model_factory = model_factory
)

In [10]:
solver = VRPSolver(
    model_config=ModelConfig(max_execution_time_minutes=0.1),
    initial_solution_builder= initial_solution_builder,
    callback=callback
)

In [11]:
sol = solver.solve(
    model_factory.build_model(routing_manager)
)

2025-08-18 11:55:35.302 | INFO     | vrp_study.initial_solution.pdptw_solution_builder:get_initial_solution:79 - (1, 32.074999999999996)
2025-08-18 11:55:35.343 | INFO     | vrp_study.initial_solution.pdptw_solution_builder:get_initial_solution:79 - (1, 16.0875)
2025-08-18 11:55:35.397 | INFO     | vrp_study.initial_solution.pdptw_solution_builder:get_initial_solution:79 - (1, 8.09375)
2025-08-18 11:55:35.443 | INFO     | vrp_study.initial_solution.pdptw_solution_builder:get_initial_solution:79 - (1, 4.096875)
2025-08-18 11:55:35.493 | INFO     | vrp_study.initial_solution.pdptw_solution_builder:get_initial_solution:79 - (1, 2.0984374999999997)
2025-08-18 11:55:35.618 | INFO     | vrp_study.initial_solution.pdptw_solution_builder:get_initial_solution:79 - (5, 1.09921875)
2025-08-18 11:55:35.733 | INFO     | vrp_study.initial_solution.pdptw_solution_builder:get_initial_solution:79 - (14, 0.599609375)
2025-08-18 11:55:35.818 | INFO     | vrp_study.initial_solution.pdptw_solution_builder:

In [12]:
# total_l = 0
# for s in sol[0]:
#     if len(s) > 0:
#         # print(s)
#         # a = routing_manager.nodes()[s[i]]
#         # b = routing_manager.nodes()[s[i+1]]
# 
#         l = sum(routing_manager.get_distance(routing_manager.nodes()[s[i]], routing_manager.nodes()[s[i + 1]]) for i in
#                 range(len(s) - 1))
#         # print(s, l)
#         total_l += l
# total_l